# RCOE GenAI Agents: Colab Deployment with Ngrok

This notebook sets up and deploys the RCOEGenAIAgents Flask application in Google Colab with ngrok public URL:
- **RCOEGenAIAgents.py** (port 5001) - MCP Architecture with Pure Gen AI Intent Routing

It handles configuration and secure credentials using `.env`, `config.properties`, and an OCI `oci_api_key.pem` file placed in the working directory.

**Security notes:**
- Do not print secrets in output cells.
- Upload `.env` and `oci_api_key.pem` at runtime; do not embed secrets in the notebook.
- ngrok tunnels are public; rotate tokens and shut down tunnels when finished.

In [ ]:
# 1) Install Required Dependencies
import sys
import subprocess

packages = [
    'flask', 'python-dotenv', 'requests', 'oci', 'pyngrok==7.1.2'
]

for pkg in packages:
    print(f"Installing {pkg}...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

print("All packages installed.")

In [ ]:
# 2) Import Libraries and Configure Environment
import os
import time
import json
import threading
import textwrap
import shutil
import signal
from pathlib import Path
from dotenv import load_dotenv
from pyngrok import ngrok, conf

WORKDIR = Path('/content/OCIGenAIBot')
WORKDIR.mkdir(parents=True, exist_ok=True)
print(f"Working directory: {WORKDIR}")

# Ensure ngrok default config doesn't reuse old tunnels in Colab sessions
conf.get_default().monitor_thread = False

## Option A: Clone from GitHub (if repo is public)

If your repository is public and contains all required files, you can clone it instead of uploading files manually. **Note:** Do NOT commit `.env` or `oci_api_key.pem` to GitHub for security reasons. You'll still need to upload those files separately.

In [ ]:
# Option A: Clone GitHub Repository (Uncomment to use)
# GITHUB_REPO = "https://github.com/Ramsiit2010/OCIGenAIBot.git"
# 
# import subprocess
# import shutil
# 
# # Clone the repo
# if Path('/content/OCIGenAIBot_clone').exists():
#     shutil.rmtree('/content/OCIGenAIBot_clone')
# 
# subprocess.check_call(['git', 'clone', GITHUB_REPO, '/content/OCIGenAIBot_clone'])
# print(f"Cloned {GITHUB_REPO}")
# 
# # Copy Python scripts and config files to WORKDIR
# clone_dir = Path('/content/OCIGenAIBot_clone')
# files_to_copy = [
#     'RCOEGenAIAgents.py',
#     'config.properties',
#     'mcp_servers/__init__.py',
#     'mcp_servers/base_server.py',
#     'mcp_servers/advisors.py',
#     'api_spec_general.json',
#     'api_spec_finance.json',
#     'api_spec_hr.json',
#     'api_spec_orders.json',
#     'api_spec_reports.json'
# ]
# 
# # Create mcp_servers directory in WORKDIR
# (WORKDIR / 'mcp_servers').mkdir(exist_ok=True)
# 
# for file_path in files_to_copy:
#     src = clone_dir / file_path
#     if src.exists():
#         if '/' in file_path:
#             dest = WORKDIR / file_path
#             dest.parent.mkdir(parents=True, exist_ok=True)
#         else:
#             dest = WORKDIR / file_path
#         shutil.copy2(src, dest)
#         print(f"Copied: {file_path}")
#     else:
#         print(f"Warning: {file_path} not found in repo")
# 
# print("\n⚠️ IMPORTANT: You still need to upload .env and oci_api_key.pem (run next cell)")

print("Option A is commented out. Uncomment the cell above to use git clone method.")

In [ ]:
# Option A (continued): Upload Secret Files (.env and oci_api_key.pem)
# Run this cell AFTER running Option A to upload sensitive files

from google.colab import files

print("Upload ONLY secret files (do NOT commit these to GitHub):")
print("- .env")
print("- oci_api_key.pem")
print()

secret_uploads = files.upload()

for name, data in secret_uploads.items():
    with open(WORKDIR / name, 'wb') as f:
        f.write(data)
    print(f"Saved: {name}")

# Set PEM permissions
pem_path = WORKDIR / 'oci_api_key.pem'
if pem_path.exists():
    os.chmod(pem_path, 0o600)
    print("✓ PEM permissions set to 600")

# Load .env
load_dotenv(dotenv_path=WORKDIR / '.env')
print("✓ .env loaded")

print("\n✓ Secret files uploaded. Ready to proceed.")

## Option B: Manual Upload (Recommended for Security)

Upload files directly to avoid exposing secrets in version control.

In [ ]:
# Option B: Manual File Upload
from google.colab import files

print("Upload ALL required files:")
print("- .env")
print("- oci_api_key.pem")
print("- config.properties")
print("- RCOEGenAIAgents.py")
print("- mcp_servers/__init__.py, mcp_servers/base_server.py, mcp_servers/advisors.py")
print("- api_spec_general.json, api_spec_finance.json, api_spec_hr.json, api_spec_orders.json, api_spec_reports.json")
print()

uploaded = files.upload()

# Create mcp_servers directory
(WORKDIR / 'mcp_servers').mkdir(exist_ok=True)

# Save uploaded files
for name, data in uploaded.items():
    # Handle nested paths (e.g., mcp_servers/base_server.py)
    if '/' in name or '\\' in name:
        file_path = WORKDIR / name.replace('\\', '/')
        file_path.parent.mkdir(parents=True, exist_ok=True)
    else:
        file_path = WORKDIR / name
    
    with open(file_path, 'wb') as f:
        f.write(data)
    print(f"Saved: {name} -> {file_path}")

# Verify critical files
required_files = ['.env', 'config.properties', 'oci_api_key.pem', 'RCOEGenAIAgents.py']
mcp_files = ['mcp_servers/__init__.py', 'mcp_servers/base_server.py', 'mcp_servers/advisors.py']
api_specs = ['api_spec_general.json', 'api_spec_finance.json', 'api_spec_hr.json', 'api_spec_orders.json', 'api_spec_reports.json']

all_required = required_files + mcp_files + api_specs
missing = [f for f in all_required if not (WORKDIR / f).exists()]

if missing:
    print(f"\n⚠️ Warning: Missing files: {missing}")
    print("RCOEGenAIAgents.py requires MCP server files and API specs to work.")
else:
    print("\n✓ All required files present")

# Set secure permissions on PEM
pem_path = WORKDIR / 'oci_api_key.pem'
if pem_path.exists():
    os.chmod(pem_path, 0o600)
    print("✓ PEM permissions set to 600")

# Load .env
load_dotenv(dotenv_path=WORKDIR / '.env')
print("✓ .env loaded")

# Preview config (no secrets)
config = {}
config_file = WORKDIR / 'config.properties'
if config_file.exists():
    with open(config_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                k, v = line.split('=', 1)
                config[k.strip()] = v.strip()
    
    region = config.get('genai_region', 'us-chicago-1')
    mock_mode = config.get('use_mock_responses', 'false')
    print(f"✓ genai_region = {region}")
    print(f"✓ use_mock_responses = {mock_mode}")
else:
    print("⚠️ config.properties not found")

## Notes on Configuration and Security

- RCOEGenAIAgents.py uses **MCP (Model Context Protocol)** architecture with 5 specialized advisors
- The application reads `genai_region` from `config.properties` and uses the local `oci_api_key.pem` from the working folder
- `.env` must contain: `OCI_USER`, `OCI_FINGERPRINT`, `OCI_TENANCY`, `OCI_KEY_FILE=oci_api_key.pem`
- Do not display or print secret values in the notebook output

In [ ]:
# 3) Set Up Ngrok (Auth Token) and Create Tunnel
from getpass import getpass

NGROK_AUTH_TOKEN = os.getenv('NGROK_AUTH_TOKEN') or getpass('Enter your ngrok auth token (input hidden): ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Ensure any previous tunnels are closed
for t in ngrok.get_tunnels():
    try:
        ngrok.disconnect(t.public_url)
    except Exception:
        pass

print("Starting ngrok tunnel for port 5001...")
http_tunnel_5001 = ngrok.connect(5001, bind_tls=True)

print("\n✓ Public URL:")
print("RCOEGenAIAgents:", http_tunnel_5001.public_url)

In [ ]:
# 4) Launch RCOEGenAIAgents Flask Application
import subprocess
import sys

process = None

os.chdir(WORKDIR)

# Start the application
script = 'RCOEGenAIAgents.py'
if not Path(script).exists():
    raise FileNotFoundError(f"{script} not found in {WORKDIR}")

env = os.environ.copy()
env['PYTHONUNBUFFERED'] = '1'
process = subprocess.Popen([sys.executable, script], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print(f"✓ Started RCOEGenAIAgents on port 5001 (PID={process.pid})")
print(f"✓ Access at: {http_tunnel_5001.public_url}")

In [ ]:
# 5) Stream Application Logs (Optional)
print("Streaming application logs for ~60 seconds. Stop the cell to end early.")
print("=" * 60)
start_time = time.time()

while time.time() - start_time < 60:
    if process.poll() is not None:
        print(f"\n⚠️ Process exited with code {process.returncode}")
        break
    
    # Non-blocking read
    line = process.stdout.readline()
    if line:
        print(line, end='')
    time.sleep(0.1)

print("\n" + "=" * 60)
print("Log tailing complete. Use the public URL above to access the app.")

In [ ]:
# 6) Health Check: Verify MCP Servers Registration
try:
    print("RCOEGenAIAgents URL:", http_tunnel_5001.public_url)
    print()
    
    # Check MCP servers registration
    import requests
    url_servers = http_tunnel_5001.public_url + '/mcp/servers'
    print(f"Checking MCP server registration at: {url_servers}")
    
    r = requests.get(url_servers, timeout=10)
    print(f"Status: {r.status_code}")
    
    if r.ok:
        data = r.json()
        print(f"\n✓ Total MCP Servers: {data.get('total', 0)}")
        print(f"✓ Gen AI Enabled: {data.get('genai_enabled', False)}")
        print(f"✓ Gen AI Region: {data.get('genai_region', 'N/A')}")
        print("\nRegistered Servers:")
        for server in data.get('servers', []):
            print(f"  - {server.get('name')}: {server.get('status')}")
    else:
        print(f"⚠️ Health check failed: {r.status_code}")
        
except Exception as e:
    print(f"⚠️ Health check error: {e}")

In [ ]:
# 7) Graceful Shutdown Helper

def stop_all():
    """Stop the application and close ngrok tunnel"""
    print("Stopping tunnel and process...")
    
    # Close ngrok tunnel
    try:
        ngrok.disconnect(http_tunnel_5001.public_url)
        ngrok.kill()
        print("✓ Ngrok tunnel closed.")
    except Exception as e:
        print(f"⚠️ Ngrok shutdown error: {e}")
    
    # Terminate Flask process
    if process:
        try:
            process.terminate()
            try:
                process.wait(timeout=5)
            except subprocess.TimeoutExpired:
                process.kill()
            print(f"✓ Terminated RCOEGenAIAgents (PID={process.pid})")
        except Exception as e:
            print(f"⚠️ Error stopping process: {e}")
    
    print("\n✓ Shutdown complete.")

print("Run stop_all() in a new cell to shut everything down when finished.")